In [239]:
import numpy as np

In [240]:
def read_input(file_name):
    with open(file_name) as f:
        elfs = []
        for y, line in enumerate(f.readlines()):
            line = line.strip()
            if line == "":
                continue
            elfs.extend((x, y) for x, c in enumerate(line) if c == "#")
    return np.array(elfs)


In [241]:
m = read_input("example")
m

array([[7, 2],
       [5, 3],
       [6, 3],
       [7, 3],
       [9, 3],
       [3, 4],
       [7, 4],
       [9, 4],
       [4, 5],
       [8, 5],
       [9, 5],
       [3, 6],
       [5, 6],
       [6, 6],
       [7, 6],
       [3, 7],
       [4, 7],
       [6, 7],
       [8, 7],
       [9, 7],
       [4, 8],
       [7, 8]])

In [242]:
m.max(axis=0)

array([9, 8])

In [243]:
def draw_map(m):
    x0, y0 = m.min(axis=0)
    x1, y1 = m.max(axis=0)
    lines = []
    for y in range(y0, y1+1):
        lines.append(['.'] * (x1 - x0 + 1))

    for x, y in m:
        lines[y - y0][x - x0] = "#"

    for line in lines:
        print("".join(line))

In [244]:
draw_map(m)

....#..
..###.#
#...#.#
.#...##
#.###..
##.#.##
.#..#..


In [245]:
moves = [
    # north
    ((0, -1), ((-1, -1), (0, -1), (1, -1))),
    # south
    ((0, 1), ((-1, 1), (0, 1), (1, 1))),
    # west
    ((-1, 0), ((-1, -1), (-1, 0), (-1, 1))),
    # east
    ((1, 0), ((1, -1), (1, 0), (1, 1)))
]    

def check_moves(elfs, moves):
    new_elfs = []
    for elf in elfs:
        for move in moves:
            new_pos = elf + move
            



In [246]:
import scipy as sp

In [247]:
dx = np.array((1, 0))
dy = np.array((0, 1))

In [248]:
# nearest neighbours of one elf
elf_num = 6
nn = m[sp.spatial.distance_matrix(m, m, p=2)[elf_num]<2]
m[elf_num], nn

(array([7, 4]),
 array([[6, 3],
        [7, 3],
        [7, 4],
        [8, 5]]))

In [249]:
# of these how are there any if we make a step?
res = nn[(sp.spatial.distance_matrix([m[elf_num]-dy], nn, p=2)<=1)[0]]
res, len(res)

(array([[6, 3],
        [7, 3],
        [7, 4]]),
 3)

In [250]:
dx = np.array((1, 0))
dy = np.array((0, 1))

moves = [
    ('north', lambda v: v - dy),
    ('south', lambda v: v + dy),
    ('west', lambda v: v - dx),
    ('east', lambda v: v + dx)
]

def try_moves(elfs, moves):
    # nearest neighbours
    dst = sp.spatial.distance_matrix(elfs, elfs, p=2)
    new_elfs = []
    for i in range(len(elfs)):
        nn = elfs[dst[i]<2]
        if len(nn) == 1:
            #print("elf %d has no neighbours and does not move" % i)
            new_elfs.append(elfs[i])
            continue
        open_direction = None
        for direction, mv in moves:
            pos = mv(elfs[i])
            nbs = nn[(sp.spatial.distance_matrix([pos], nn, p=2)<=1)[0]]
            if len(nbs) == 1:
                open_direction = direction
                break
        if open_direction:
            #print("Elf %d can move %s" % (i, open_direction))
            new_elfs.append(pos)
        else:
            new_elfs.append(elfs[i])
            #print("Elf %d cannot move" % (i))
    return np.array(new_elfs)


In [251]:
m2 = try_moves(m, moves)

# find the ones who will end up in the same position
mask = np.sum(sp.spatial.distance_matrix(m2, m2, p=2) == 0, axis=1) > 1
# they will not move!
m2[mask] = m[mask]

In [252]:
draw_map(m2)

.....#...
...#...#.
.#..#.#..
.....#..#
..#.#.##.
#..#.#...
#.#.#.##.
.........
..#..#...


In [253]:
#moves = moves[1:] + [moves[0]]

In [254]:
def take_moves(elfs, moves, num_rounds=10):
    moves = list(moves) # take copy
    m = elfs
    for i in range(num_rounds):
        print(i, moves[0])
        m2 = try_moves(m, moves)

        # find the ones who will end up in the same position
        mask = np.sum(sp.spatial.distance_matrix(m2, m2, p=2) == 0, axis=1) > 1
        # they will not move!
        m2[mask] = m[mask]

        moves = moves[1:] + [moves[0]]
        m = m2
    return m

In [255]:
m_10 = take_moves(m, moves, num_rounds=10)
draw_map(m_10)

0 ('north', <function <lambda> at 0x7ff761afbaf0>)
1 ('south', <function <lambda> at 0x7ff761afbca0>)
2 ('west', <function <lambda> at 0x7ff761afbc10>)
3 ('east', <function <lambda> at 0x7ff761afb5e0>)
4 ('north', <function <lambda> at 0x7ff761afbaf0>)
5 ('south', <function <lambda> at 0x7ff761afbca0>)
6 ('west', <function <lambda> at 0x7ff761afbc10>)
7 ('east', <function <lambda> at 0x7ff761afb5e0>)
8 ('north', <function <lambda> at 0x7ff761afbaf0>)
9 ('south', <function <lambda> at 0x7ff761afbca0>)
......#.....
..........#.
.#.#..#.....
.....#......
..#.....#..#
#......##...
....##......
.#........#.
...#.#..#...
............
...#..#..#..


In [256]:
x0, y0 = m_10.min(axis=0)
x1, y1 = m_10.max(axis=0)
(x1-x0)*(y1-y0)

110

# Part 1

In [257]:
m = read_input("input")

In [258]:
m_10 = take_moves(m, moves, num_rounds=10)

0 ('north', <function <lambda> at 0x7ff761afbaf0>)
1 ('south', <function <lambda> at 0x7ff761afbca0>)
2 ('west', <function <lambda> at 0x7ff761afbc10>)
3 ('east', <function <lambda> at 0x7ff761afb5e0>)
4 ('north', <function <lambda> at 0x7ff761afbaf0>)
5 ('south', <function <lambda> at 0x7ff761afbca0>)
6 ('west', <function <lambda> at 0x7ff761afbc10>)
7 ('east', <function <lambda> at 0x7ff761afb5e0>)
8 ('north', <function <lambda> at 0x7ff761afbaf0>)
9 ('south', <function <lambda> at 0x7ff761afbca0>)


In [259]:
draw_map(m_10)

.........#................................................................#.........
........#..#......#..............#..#.....#..#.....#...#.##.....##..#...............
....##........##....#.##..#........#............##.........#...........#..#.....#...
.........#...#...#............#......##...#............#..#..##..#...#....##.#......
......##..#..#...#......##..#.......#...##..#.#.#.#....#..#..##.....#..#............
.........#..#.....##.##......#..###...#.##....##....###..#...#..#.#..#...#.#.#.#.#..
..#..#..#.##.#.##.#.#...#..#...#.#..##....#.#....#..#...#..#.#...#.#...#.##.........
......#...#.#.#........#.#.##.#........#..#.#.#..#...#.#.#..#.###.#.#.#..#..#..#....
........##.#.#.#.#.#.#..#....#.#.##.#.#..#.##.##.##.#.#.#.#..##..#.#.#.##.#...#.....
....###...#.#.#.#.#.#.##.##.#......#...#..##..##...#.###.#.#.#..#.#.#.#..#.#..#..#..
.....#..##.##..#.#.#.#..#..#.#..#.#.#.#.##...#..#.#.#.#.#.#.#..###...#.#.#..#.#.....
...#..#...#..#..#.#.#..#.#..#.#..#.#.#..########.#.#.#.#.#.#.##..

In [260]:
x0, y0 = m_10.min(axis=0)
x1, y1 = m_10.max(axis=0)
(x1-x0)*(y1-y0)

6889

```
That's not the right answer; your answer is too high. If you're stuck, make sure you're using the full input data; there are also some general tips on the about page, or you can ask for hints on the subreddit. Please wait one minute before trying again. (You guessed 6889.) [Return to Day 23]
```